# Darks Souls II Reviews (2025)

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import altair as alt
import re

## Steam Reviews as of 3/30/24:

In [2]:
df = pd.read_csv('reviews.csv')
reviews = df.copy()
reviews = reviews.set_index('recommendationid')
reviews.drop(columns={'Unnamed: 0', 'in_early_access'}, inplace=True)

Converting date of review from unix:

In [30]:
reviews['month_name'] = pd.to_datetime(reviews.update_date, unit='s').dt.month_name()
reviews['month']      = pd.to_datetime(reviews.update_date, unit='s').dt.month
reviews['year']       = pd.to_datetime(reviews.update_date, unit='s').dt.year
reviews['day']        = pd.to_datetime(reviews.update_date, unit='s').dt.day

C:\Users\bdion\AppData\Local\Temp\ipykernel_13528\2518063978.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviews['month_name'] = pd.to_datetime(reviews.update_date, unit='s').dt.month_name()
C:\Users\bdion\AppData\Local\Temp\ipykernel_13528\2518063978.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviews['month'] = pd.to_datetime(reviews.update_date, unit='s').dt.month
C:\Users\bdion\AppData\Local\Temp\ipykernel_13528\2518063978.py:3: SettingWithCopyWarning: 
A value is trying to be set on a c

Focusing on just the English reviews:

In [4]:
reviews = reviews[reviews.language == 'english']

## Cleaning up the reviews

In [ ]:
reviews = reviews.dropna(subset=['review'])
reviews.shape

(45563, 7)

In [ ]:
reviews['review'] = reviews.review.str.lower()

In [ ]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

stop_words = set(stopwords.words('english'))
def remove_stopwords(text):
    words = word_tokenize(text)
    return ' '.join([word for word in words if word.lower() not in stop_words])

In [ ]:
reviews['review'] = reviews.review.apply(remove_stopwords)

In [ ]:
# Removing urls:
r = [re.sub(r'http\S+', '', review).lower().strip() if pd.notna(review) else review for review in reviews.review]

# Removing esc sequences, punctuation, and numbers:
    # There's some ASCII art in some of the reviews
r = [re.sub(r'[^a-z]', ' ', review).strip() if pd.notna(review) else review for review in r]

In [ ]:
# Removing multiple and trailing whitespaces:
r = [re.sub(r' +', ' ', review).strip() if pd.notna(review) else review for review in r]

In [ ]:
reviews['review'] = r

In [7]:
reviews

,review,language,init_date,update_date,voted_up,month,year
recommendationid,,,,,,,
190511148,dont give up skeleton,english,1742267451,1742267451,True,March,2025
190504311,boia,english,1742259081,1742259081,True,March,2025
190502415,i love this game to pieces it s the worst soul...,english,1742256864,1742256864,True,March,2025
190501465,i probably shouldn t recommend it because of h...,english,1742255757,1742255757,True,March,2025
190500200,peak souls,english,1742254339,1742254339,True,March,2025
...,...,...,...,...,...,...,...
15162268,try tongue but hole,english,1427932431,1428081346,True,April,2015
15162220,so far so good played it for mins so far with ...,english,1427932153,1427932153,True,April,2015
15162161,still haven t died bonedrinker rufus keep the ...,english,1427931845,1427931845,True,April,2015


For sake of analysis specifically on the actual reviews, drop any rows that have no reviews:

In [8]:
reviews = reviews.dropna(subset=['review'])
reviews.shape

(45563, 7)

In [9]:
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [10]:
tfidf = TfidfVectorizer(sublinear_tf=True,
                        analyzer='word',
                        max_features=4000,
                        tokenizer=word_tokenize,
                        stop_words=stopwords.words("english"))

In [11]:
review_txt = reviews.review.values.flatten()
tfidf_array = tfidf.fit_transform(review_txt).toarray()
tfidf_df = pd.DataFrame(tfidf_array)
tfidf_df.columns = tfidf.get_feature_names_out()

c:\Users\bdion\anaconda3\Lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
c:\Users\bdion\anaconda3\Lib\site-packages\sklearn\feature_extraction\text.py:408: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'d", "'ll", "'re", "'s", "'ve", 'could', 'might', 'must', "n't", 'need', 'sha', 'wo', 'would'] not in stop_words.
  warnings.warn(


- Most common word among the reviews isn't very informative - including some of the other popular words
    - Looking at subsets of the reviews could be useful

### Topic Modeling:
- Exploring certain aspects on why people like the game
    - Also get critiques of the game in positive reviews (if any but there sure is considering DS2's reputation in the community)

- Exploring why people don't like the game:
    - Also get positive aspects within this subset of the reviews
    
- Algorithms I can use to perform topic modeling:
    1. Latent Dirichlet Allocation (LDA) 
    2. Non-negative Matrix Factorization (NMF)

Splitting the reviews by how many do and don't recommend buying the game:

In [12]:
pos_reviews = reviews[reviews['voted_up'] == True]
neg_reviews = reviews[reviews['voted_up'] == False]

In [13]:
pos_reviews.shape, neg_reviews.shape

((37387, 7), (8176, 7))